In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ea-stocks-latest/EA_stock_split.csv
/kaggle/input/ea-stocks-latest/EA_stock_dividend.csv
/kaggle/input/ea-stocks-latest/EA_stock_price.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import altair as alt
import plotly.graph_objects as go
import prophet as Prophet
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


stock_price_df = pd.read_csv('/kaggle/input/ea-stocks-latest/EA_stock_price.csv')
dividend_df = pd.read_csv('/kaggle/input/ea-stocks-latest/EA_stock_dividend.csv')
stock_split_df = pd.read_csv('/kaggle/input/ea-stocks-latest/EA_stock_split.csv')

print(stock_price_df.info())
print(dividend_df.info())
print(stock_split_df.info())

print(stock_price_df.isnull().sum())
print(stock_price_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6258 entries, 0 to 6257
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         6258 non-null   object 
 1   Open Price   6258 non-null   float64
 2   High Price   6258 non-null   float64
 3   Low Price    6258 non-null   float64
 4   Close Price  6258 non-null   float64
 5   Volume       6258 non-null   float64
dtypes: float64(5), object(1)
memory usage: 293.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              16 non-null     object 
 1   Declaration Date  15 non-null     object 
 2   Record Date       15 non-null     object 
 3   Payment Date      15 non-null     object 
 4   Dividend          16 non-null     float64
dtypes: float64(1), object(4)
memory usage: 768.0+ bytes
None
<class

**Step 2: Data Cleaning and Preprocessing**

**Step 3: Exploratory Data Analysis (EDA)**

In [3]:
stock_price_df['Price Volatility'] = stock_price_df['Close Price'].rolling(window=30).std()

fig = go.Figure()

fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=stock_price_df['Close Price'],
                         mode='lines', name='Close Price', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=stock_price_df['Price Volatility'],
                         mode='lines', name='Volatility (30-day)', line=dict(color='red', dash='dash')))

fig.update_layout(title='EA Stock Price and Volatility (30-day Rolling)',
                  xaxis_title='Date', yaxis_title='Price / Volatility',
                  template='plotly_dark')

fig.show()

> Dividend Payment Trend with Altair

In [4]:
dividend_df['Date'] = pd.to_datetime(dividend_df['Date'])

# An interactive line chart for dividends over time
dividend_chart = alt.Chart(dividend_df).mark_line(point=True).encode(
    x='Date:T',
    y='Dividend:Q',
    tooltip=['Date:T', 'Dividend:Q']
).properties(
    title='EA Dividend Payments Over Time',
    width=700,
    height=400
).interactive()

dividend_chart.display()

alt.Chart(...)

> Heatmap for Stock Price Correlations

In [5]:
corr_matrix = stock_price_df[['Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume']].corr()

fig = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='RdBu_r', aspect='auto')

fig.update_layout(title='EA Stock Price Feature Correlation Matrix',
                  template='plotly_dark')

fig.show()

**Step 4: Financial Analysis**

> Technical Analysis:
Calculate Moving Averages (Simple and Exponential) and plot them to smooth stock price data and identify trends.

Calculate and plot Relative Strength Index (RSI) to evaluate overbought/oversold conditions.

> Event Study (Stock Splits):
Analyze the impact of stock split announcements on EA’s stock price by conducting an event study.

Compare pre- and post-split stock performance using cumulative abnormal returns.

In [6]:
fig = go.Figure(data=[go.Candlestick(x=stock_price_df['Date'],
                                     open=stock_price_df['Open Price'],
                                     high=stock_price_df['High Price'],
                                     low=stock_price_df['Low Price'],
                                     close=stock_price_df['Close Price'],
                                     increasing_line_color='green',
                                     decreasing_line_color='red')])

# Calculate 50-day and 200-day Moving Averages
stock_price_df['50-day MA'] = stock_price_df['Close Price'].rolling(window=50).mean()
stock_price_df['200-day MA'] = stock_price_df['Close Price'].rolling(window=200).mean()


fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=stock_price_df['50-day MA'],
                         mode='lines', name='50-day MA', line=dict(color='blue', width=1.5)))
fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=stock_price_df['200-day MA'],
                         mode='lines', name='200-day MA', line=dict(color='orange', width=1.5)))

fig.update_layout(title='EA Stock Price Candlestick with Moving Averages',
                  xaxis_title='Date', yaxis_title='Stock Price ($)',
                  template='plotly_dark')

fig.show()


# RSI Calculation (Relative Strength Index)
window_length = 14

# Calculate the difference in prices
delta = stock_price_df['Close Price'].diff()

# Calculate gains and losses
gain = (delta.where(delta > 0, 0)).rolling(window=window_length).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=window_length).mean()

# Calculate RSI
rs = gain / loss
stock_price_df['RSI'] = 100 - (100 / (1 + rs))

# Plot RSI
fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=stock_price_df['RSI'], mode='lines', name='RSI'))

fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=[70]*len(stock_price_df), mode='lines', name='Overbought (70)', line=dict(dash='dash', color='red')))
fig.add_trace(go.Scatter(x=stock_price_df['Date'], y=[30]*len(stock_price_df), mode='lines', name='Oversold (30)', line=dict(dash='dash', color='green')))

fig.update_layout(title='EA Stock RSI with Interactive Plotly',
                  xaxis_title='Date', yaxis_title='RSI',
                  template='plotly_dark')

fig.show()

**Step 5: Modeling and Prediction**

> Prophet Model for Time Series Forecasting

In [7]:
from prophet import Prophet

prophet_df = stock_price_df[['Date', 'Close Price']].rename(columns={'Date': 'ds', 'Close Price': 'y'})

model = Prophet()
model.fit(prophet_df)

future = model.make_future_dataframe(periods=365)
forecast = model.predict(future)

fig = go.Figure()

# Historical data
fig.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['y'], mode='lines', name='Historical Data'))

# Forecast data
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))

fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill=None, mode='lines', name='Upper Confidence Interval', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='lines', name='Lower Confidence Interval', line=dict(dash='dash')))

fig.update_layout(title='EA Stock Price Forecast Using Prophet and Plotly',
                  xaxis_title='Date', yaxis_title='Stock Price ($)',
                  template='plotly_dark')

fig.show()


13:45:46 - cmdstanpy - INFO - Chain [1] start processing
13:45:50 - cmdstanpy - INFO - Chain [1] done processing
